# RAIL SOMPZ Estimator Demo

Authors: Sam Schmidt, Justin Myles

Last successfully run: June 12, 2024

This demo notebook follows the informer demo for the `rail_sompz` method, `rail_sompz_inform_demo.ipynb`, and uses the model file `DEMO_romandesc_model.pkl` that is created in that notebook.  So, you will need to run that notebook and train a model consisting of the wide and deep SOMs before this one in order to run the estimate stage and produce tomographic bin estimates.

The estimate method uses the two SOMs trained in the inform method in order to construct tomographic bin estimates.  The algorithm works by determining weights for a spectroscopic dataset based on a wider "deep" dataset relative to a (usually larger) wide dataset.  See [Myles, Alarcon et al. 2021](https://arxiv.org/pdf/2012.08566) and references in [Campos et al. 2023](https://github.com/AndresaCampos/sompz_y6) for more details on the method.

We'll start with our usual imports:

In [ ]:
import os
import sys
import numpy as np
from rail.core.utils import RAILDIR
from rail.core import common_params
import tables_io
import matplotlib.pyplot as plt

import pandas as pd
import astropy.io.fits as fits

In [ ]:
from rail.estimation.algos.sompz import SOMPZInformer
from rail.estimation.algos.sompz import SOMPZEstimator

The SOMPZ method usually leverages a "deep" dataset with extra bands (often in the near-infrared), where the extra photometric information in the extended wavelength coverage enables a magnitudes/colors -> redshift mapping with less degeneracies than when using optical colors along.  For this demo, we will use data from the Rubin-Roman simulation [Citation needed!], which does contain simluated photometry for both the Rubin optical `ugrizy` bands as well as the Roman `JHFK` bands.  We have included a command-line tool in RAIL that will grab several data files that we will use in this demo.  If you ran the informer demo they are already in place and you can ignore the following cell, if you moved/deleted files, or just copied the model from the informer stage and still need the data, then uncomment the lines in the cell below to grab the data files, move, and untar them in the appropriate location.

In [ ]:
#!curl -O https://portal.nersc.gov/cfs/lsst/PZ/roman_desc_demo_data.tar.gz
#!mkdir DEMODATA
#!tar -xzvf roman_desc_demo_data.tar.gz
#!mv romandesc*.hdf5 DEMODATA/

Now, let's load the three files that we will use into memory.  The "spec" file contains the galaxies with spectroscopic redshifts, these are usually a subset of the "deep" data (and that is the case here).  The "deep" data contains both optical and NIR bands, in this case `ugrizyJHF`.  And the "wide" data contains only `ugrizy` photometry.  The code will determine the cell occupation of the spec sample, determine weights via the deep sample, and attempt to create tomographic bin estimates for the sample based on SOM cell occupation.


There are two sets of files included in the Rubin-Roman download, one set that is a factor of 20 larger than the other.  For a quick demo, use the file names for `specfile`, `deepfile`, and `widefile` as-is below, for a more robust estimate with more training and estimation data, switch to the larger files by uncommenting and commenting the file names below:

In [ ]:
from rail.core.data import TableHandle
from rail.core.stage import RailStage

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
#
## Larger files to use if you want slightly more robust demo (will take longer to run)
#specfile = "./DEMODATA/romandesc_spec_data_37k_noinf.hdf5"
#deepfile = "./DEMODATA/romandesc_deep_data_75k_noinf.hdf5"
#widefile = "./DEMODATA/romandesc_wide_data_100k_noinf.hdf5"
## smaller files for a quick demo, swap which lines are commented if you don't mind some extra run time
specfile = "./DEMODATA/romandesc_spec_data_18c_noinf.hdf5"
deepfile = "./DEMODATA/romandesc_deep_data_37c_noinf.hdf5"
widefile = "./DEMODATA/romandesc_wide_data_50c_noinf.hdf5"

spec_data = DS.read_file("spec_data", TableHandle, specfile)
balrog_data = DS.read_file("deep_data", TableHandle, deepfile)
wide_data = DS.read_file("wide_data", TableHandle, widefile)

We need to set up several parameters used by the estimate stage, namely the names of the inputs (for both deep and wide), the names of the input errors (again for both deep and wide), the zero points.  In our dataset, the bands are simply called e.g. `u`, and `J`, and the errors `u_err` and `J_err`.  The "deep" SOM we will use both optical and NIR bands, for the wide data we will only use ugrizy: 

In [ ]:
bands = ['u','g','r','i','z','y','J','H', 'F']
#bands = ['u','g','r','i','z','y']

deepbands = []
deeperrs = []
zeropts = []
for band in bands:
    deepbands.append(f'{band}')
    deeperrs.append(f'{band}_err')
    zeropts.append(30.)

widebands = []
wideerrs = []  
for band in bands[:6]:
    widebands.append(f'{band}')
    wideerrs.append(f'{band}_err')


In [ ]:
print(widebands)

There are many configuration parameters that we can access to control the behavior of the estimate stage, these are described below.  Any values not specified will take on their default values as set in the parameter config that is located in the class:

`bin_edges`: the list of edges of tomo bins<br>
`zbins_min`: minimum redshift for output grid<br>
`zbins_max`: maximum redshift for output grid<br>
`zbins_dz`: delta z for defining output grid<br>
`spec_groupname`: hdf5_groupname for spec_data<br>
`balrog_groupname`: hdf5_groupname for balrog_data<br>
`wide_groupname`: hdf5_groupname for wide_data<br>
`specz_name`: column name for true redshift in specz sample<br>
`inputs_deep`: list of the names of columns to be used as inputs for deep data<br>
`input_errs_deep`: list of the names of columns containing errors on inputs for deep data<br>
`inputs_wide`: list of the names of columns to be used as inputs for wide data<br>
`input_errs_wide`: list of the names of columns containing errors on inputs for wide data<br>
`zero_points_deep`: zero points for converting mags to fluxes for deep data, if needed<br>
`zero_points_wide`: zero points for converting mags to fluxes for wide data, if needed<br>
`som_shape_deep`: shape for the deep som, must be a 2-element tuple<br>
`som_shape_wide`: shape for the wide som, must be a 2-element tuple<br>
`som_minerror_deep`: floor placed on observational error on each feature in deep som<br>
`som_minerror_wide`: floor placed on observational error on each feature in wide som<br>
`som_wrap_deep`: flag to set whether the deep SOM has periodic boundary conditions<br>
`som_wrap_wide`: flag to set whether the wide SOM has periodic boundary conditions<br>
`som_take_log_deep`: flag to set whether to take log of inputs (i.e. for fluxes) for deep som<br>
`som_take_log_wide`: flag to set whether to take log of inputs (i.e. for fluxes) for wide som<br>
`convert_to_flux_deep`: flag for whether to convert input columns to fluxes for deep data, set to true if inputs are mags and to False if inputs are already fluxes<br>
`convert_to_flux_wide`=Param(bool, False, msg="flag for whether to convert input columns to fluxes for wide data<br>
`set_threshold_deep`: flag for whether to replace values below a threshold with a set number<br>
`thresh_val_deep`: threshold value for set_threshold for deep data<br>
`set_threshold_wide`: flag for whether to replace values below a threshold with a set number<br>
`thresh_val_wide`: threshold value for set_threshold for wide data<br>
`debug`: boolean reducing dataset size for quick debugging, will take only the first 200 rows of each of the spec, deep, and wide files so things run quicker<br>


Let's define a dictionary with the config parameters that we need to set in order to have things work with the Roman-DESC data.  We will also set a custom set of `bin_edges` with six values that will produce five tomographic bins where the SOM will do its best to assign galaxies to the bins bounded by each of the bin edges.

In [ ]:
tomo_binedges = [0.0, 0.5, 0.8, 1.1, 1.5, 3.0]
som_params = dict(inputs_deep=deepbands, input_errs_deep=deeperrs, 
                  inputs_wide=widebands, input_errs_wide=wideerrs,  
                  zero_points_deep=zeropts, zero_points_wide=zeropts[:6],
                  convert_to_flux_deep=True, convert_to_flux_wide=True, 
                  set_threshold_deep=True, thresh_val_deep=1.e-5, 
                  som_shape_wide=(25, 25), som_minerror_wide=0.005,
                  som_take_log_wide=False, som_wrap_wide=False,
                  zbins_min=0.0, zbins_max=5.0, zbins_dz=0.05,
                  spec_groupname='', balrog_groupname='', wide_groupname='',
                  bin_edges=tomo_binedges)

Now we will set up the stage that will run the estimator, we will grab the model file that we trained in the informer demo run previously:

In [ ]:
rd_som_estimate = SOMPZEstimator.make_stage(name="som_estimator", 
                                         model="DEMO_romandesc_model.pkl", 
                                         **som_params)

Now, let's run the estimator:

In [ ]:
%%time
rd_som_estimate.estimate(spec_data,
                         balrog_data,
                         wide_data)

The five tomographic bin estimates are stored in an output file with the name that we assigned to the SOMPZEstimator stage, prepended with an `nz_`, let's read in that file and display our tomographic bin estimates, along with the bin edges that we set:

In [ ]:
import qp

In [ ]:
ens = qp.read("nz_som_estimator.hdf5")

In [ ]:
binedges = [0.0, 0.5, 0.8, 1.1, 1.5, 3.0]
fig, axs = plt.subplots(1,1, figsize=(10,6))
cols=['r','purple','b','orange','k']
for i, col in enumerate(cols):
    ens[i].plot_native(axes=axs, color=col)
    axs.axvline(binedges[i+1], color=col, ls='--', lw=0.9)
axs.set_xlabel("redshift", fontsize=14)
axs.set_ylabel("N(z)", fontsize=14)
axs.set_xlim(0,3.25)

Looks very good! Nice separation, without many bumps outside of the bin due to degeneracies.  The addition of the near-infrared bands can break many of the degeneracies where the Lyman and Balmer breaks are confused for each other.  This demonstrates the power of this technique, and how using NIR (or any other additional band information) can help us in determining our redshift distributions.

We can also try breaking up our wide sample into tomo bins based on the "best" cell assignment and compare our estimated redshift distribution to the histogram of wide sample redshifts in that bin.  The estimator will spit out a file named `tomo_bin_mask_wide_data_[name of stage].hdf5`, in this case `tomo_bin_mask_wide_data_som_estimator.hdf5`.  That file contains a dictionary with two keys, `bin` which is an integer corresponding to which tomographic bin each galaxy is assigned to, and `weight`, a weight (in this case they are all 1.0) for each galaxy within the bin.  We can use this data to compare our tomographic bin estimate to the true redshift distributions for our bin samples.

In [ ]:
tomo_mask_file = "tomo_bin_mask_wide_data_som_estimator.hdf5"
tomomaskdata = tables_io.read(tomo_mask_file)
tomo_mask = tomomaskdata['bin']
tomo_weight = tomomaskdata['weight']

In [ ]:
sz = wide_data()['redshift']
nbins = len(tomo_binedges)-1

Let's make some comparison plots:

In [ ]:
histedges = np.linspace(-.05,5.05,208)
fig, axs = plt.subplots(5, 1, figsize=(10,12))
for i in range(nbins):
    binmask = (tomo_mask == i)
    binsz = sz[binmask]
    axs[i].hist(binsz, bins=histedges, color='k', alpha=0.4, density=True, label='true redshift hist')
    ens[i].plot_native(axes=axs[i], color='r', label='SOMPZ tomo estimate')
    axs[i].set_ylabel("N(z)", fontsize=12)
    axs[i].set_xlim(-.005,3.15)
axs[4].set_xlabel("redshift", fontsize=12)
axs[0].legend(loc='upper right', fontsize=14)
#plt.savefig("tomo_bins_truth_compare.jpg", format="jpg")

We see very good agreement between the true distributions and our SOMPZ estimates, including the small outlier bumps in the highest redshift bin.